<a href="https://colab.research.google.com/github/Shatugambo08/RESGAD-1/blob/main/Analyzing_Cancellation_rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analyzing Trip cancellation rate

### Overview of the project

This query calculates the cancellation rate of trips of each day from 10-01-2013 to 10-03-2013 using SQL server.

We will be considering trips where both client and driver are not banned . Using this procedure joining_data---Filtering_data---Calculating_ Cancellation_rate---Grouping data--Ordering_result.

In [ ]:
---Create Table Trips
CREATE TABLE Trips (
    id INT PRIMARY KEY,
    client_id INT,
    driver_id INT,
    city_id INT,
    status VARCHAR(20),
    request_at VARCHAR(25),
    FOREIGN KEY (client_id) REFERENCES Users(users_id),
    FOREIGN KEY (driver_id) REFERENCES Users(users_id)
);

SyntaxError: invalid syntax (ipython-input-2-2773720026.py, line 1)

In [ ]:
---Create Table Users
CREATE TABLE Users (
    users_id INT PRIMARY KEY,
    banned VARCHAR(10),
    role VARCHAR(10)
);

In [ ]:
---Populate table Trips
INSERT INTO Trips (id, client_id, driver_id, city_id, status, request_at) VALUES
(1, 1, 2, 1, 'completed', '2013-10-01'),
(2, 1, 2, 1, 'cancelled_by_client', '2013-10-01'),
(3, 1, 2, 1, 'completed', '2013-10-01'),
(4, 1, 2, 1, 'completed', '2013-10-02'),
(5, 1, 2, 1, 'completed', '2013-10-02'),
(6, 1, 2, 1, 'completed', '2013-10-03'),
(7, 1, 2, 1, 'cancelled_by_driver', '2013-10-03');

In [ ]:
---Populate table Users
INSERT INTO Users (users_id, banned, role) VALUES
(1, 'No', 'client'),
(2, 'No', 'driver'),
(3, 'Yes', 'client'),
(4, 'No', 'driver');

In [ ]:
FROM Trips t
INNER JOIN Users c ON t.client_id = c.users_id
INNER JOIN Users d ON t.driver_id = d.users_id

---Joining data.

Using INNER JOIN cause every trip should have a client and driver,using trips where both not banned.

In [ ]:
WHERE CAST(t.request_at AS DATE) BETWEEN '2013-10-01' AND '2013-10-03'
AND c.banned = 'No'
AND d.banned = 'No'

--Filtering_data

Filter trips between '2013-10-01' and '2013-10-03'. Since request_at is a varchar, cast it to DATE for reliable comparison in SQL Server

In [ ]:
SUM(CASE WHEN t.status IN ('cancelled_by_client', 'cancelled_by_driver') THEN 1 ELSE 0 END)
CAST(...) AS DECIMAL) / COUNT(*)
ROUND(..., 2)

---Cancellation_Rate

To get Cancelled Trips,Count trips where status is 'cancelled_by_client' or 'cancelled_by_driver' using a CASE statement within SUM.

Use COUNT(*) to get the total number of trips per day.

Divide cancelled trips by total trips.

Cast the numerator to DECIMAL to avoid integer division (which would truncate the result to 0).

Use ROUND to format the result to two decimal places.

In [ ]:
SELECT
    CAST(t.request_at AS DATE) AS Day,
    ROUND(CAST(SUM(CASE WHEN t.status IN ('cancelled_by_client', 'cancelled_by_driver') THEN 1 ELSE 0 END) AS DECIMAL) / COUNT(*), 2) AS [Cancellation Rate]
GROUP BY CAST(t.request_at AS DATE)

--- Grouping_data.

Group results by the date, casting request_at to DATE.


In [ ]:
ORDER BY Day

Ordering_result

Order by Day ensure results are presented chronologically, using the exact date given (2013-10-01 to 2013-10-03).

In [ ]:
SELECT
    CAST(t.request_at AS DATE) AS Day,
    ROUND(CAST(SUM(CASE WHEN t.status IN ('cancelled_by_client', 'cancelled_by_driver') THEN 1 ELSE 0 END) AS DECIMAL) / COUNT(*), 2) AS [Cancellation Rate]
FROM Trips t
INNER JOIN Users c ON t.client_id = c.users_id
INNER JOIN Users d ON t.driver_id = d.users_id
WHERE CAST(t.request_at AS DATE) BETWEEN '2013-10-01' AND '2013-10-03'
    AND c.banned = 'No'
    AND d.banned = 'No'
GROUP BY CAST(t.request_at AS DATE)
ORDER BY Day

--- Merging data